# Exploring Data from a Video Game Research Server

In [ ]:
### Please run thsi cell before moving forward:
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)
library(GGally)
options(repr.matrix.max.rows = 6)

In [ ]:
players_data <- read.csv("data/players.csv")
# head(players_data)

In [ ]:
seasons_data <- read.csv("data/sessions.csv")
# head(seasons_data)

## Data Description

#### I have chosen to work with the players dataset provided from "players.csv" 

In [ ]:
number_observations <- nrow(players_data)
number_variables    <- ncol(players_data)
dimensions_players <- cat("Observations:", number_observations, 
                          " Variables:", number_variables)


- There are only 196 players, giving us only 196 observations to work with. **A stratified approach to training and validation might be best (i.e. 5-fold cross-validation)**.

- Two of the 7 variables provided will most likely **not be used** for any statistical analysis and modelling regardless of the question chosen. These are:
    1.  `hashedEmail`
    2.  `name`

In [ ]:
statistical_summary_played_hours <- players_data |> 
    summarize(
        played_hours_mean   = round(mean(played_hours, na.rm = TRUE), 2),
        played_hours_median = round(median(played_hours, na.rm = TRUE), 2),
        played_hours_min    = round(min(played_hours, na.rm = TRUE), 2),
        played_hours_max    = round(max(played_hours, na.rm = TRUE), 2))

played_none <- players_data |>
    filter(played_hours == 0) |>
    summarize(count = n(), .groups = "drop")

statistical_summary_played_hours
played_none

**Played Hours Numeric Summary:**
- Based on the values provided by the statistical summary, the data show a **right-skewed distribution**. Looking further into the dataset and calculating the number of players who played 0 hours, I suspect the reason for the right-skew is that a rather large number of players **(85 players, 43.37%) have not played the game and are included in the study**.
- This prompt me to invistage this idea via a **histogram visualization**.
- This observation also suggests that it might be interesting to ask questions about **what characteristics predict players whom might contribute significantly to the project** - assuming that this factor, **large amount of data contribution/player**, produces better results for the Pacific Team. (relating to **Question 2**)

In [ ]:
statistical_summary_age <- players_data |> 
    summarize(
        age_mean            = round(mean(Age, na.rm = TRUE), 2),
        age_median          = round(median(Age, na.rm = TRUE), 2),
        age_min             = round(min(Age, na.rm = TRUE), 2),
        age_max             = round(max(Age, na.rm = TRUE), 2)) 
statistical_summary_age

**Age Numeric Summary:**

- The study has some very young and very old players (**outliers**) but overall the age group for most participants is late teens to early twenties.


In [ ]:
experience_count_per <- players_data |>
    group_by(experience) |>
    summarize(count = n(), .groups = "drop") |>
    mutate(per = round(100 * count / sum(count), 2))

experience_count_per

**Experience - Categorical Composition:**

- There is a reasonable spread of experience with the smallest class being "Pro" at around 7%. **A great question and experiement to do would be to see the contribution of each class to play time and their newsletter subscription status**.

In [ ]:
gender_count_per <- players_data |>
    group_by(gender) |>
    summarize(count = n(), .groups = "drop") |>
    mutate(per = round(100 * count / sum(count), 2))

gender_count_per

**Gender - Categorical Composition:**

- There is a **strong class imbalance with "Male" category** coming at about **63.27%**. Given what we know about how many players have played 0 hours and the rather large imbalance towards "Male", **it's imparative to check what the contributation of this class to the study per instance**. *My hypothesis is that many male players have signed up but did not engage with the game and study suggesting an explanation to the large 0 hours played spike*.
- The category is also **noisy due to it being self-reported** and given that "Other" and "Prefer not to say" are present instead of contributing to the present categories or contributing to gender categories not represented in the population on the surface.

In [ ]:
subscription_status <- players_data |>
    group_by(subscribe) |>
    summarize(count = n(), .groups = "drop") |>
    mutate(per = round(100 * count / sum(count), 2))

subscription_status

**Subscription Status - Categorical Composition:**

- There is a **strong imbalance towards "TRUE"** which suggests a strong interest in the experiment. However, **since subscription status can be thought of as a proxy for interest, one should examine the relationship between subscription status and contribution to the experiment (playing time)** as you would *expect* newsletter subscribers to have higher playing hours on average.

### Seasons Data Data Descriptions

In [ ]:
sd_number_observations <- nrow(seasons_data)
sd_number_variables    <- ncol(seasons_data)
sd_dimensions_players <- cat("Observations:", sd_number_observations, 
                          " Variables:", sd_number_variables)


In [ ]:
sessions_per_player <- seasons_data |>
  count(hashedEmail, name = "sessions")

# head(sessions_per_player)

In [ ]:
sessions_per_player_summary <- sessions_per_player |>
  summarize(
    users  = n(),
    min    = min(sessions),
    median = median(sessions),
    mean   = round(mean(sessions), 2),
    max    = max(sessions)
  )

sessions_per_player_summary

## Questions

**The Questions:**

- **Broad Question (#2)**: We would like to know which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts.
- **Specific question: Can player's age, gender, and experience predict whether they are in the top 25% of total play time?** (whether they are going to contribute significantly to the experiement or not.)

**More About The Choice:**

- Since I saw that the newsletter subscribers category is **not a good proxy of contribution and is rather a proxy for interest**. I wanted to examine how the experiments stakeholders can **improve their recruitment practices to get more players who can contribute more significantly to their project without inducing bias** to their selection process.
- I have chosen top 25% as to present this data into **four quartiles**, this makes it easier to communicate the significance of the result (if any) to the stakeholders. I also cannot chose a random cuttoff for played_time (say at an hour) as it has no innate meaning neither does it relate to the dataset in a meaningful manner.

**How The Data Helps Answer The Question:**

- Due to proposing using quartile-based definition for heavy contributors, (with heavy contributor defined as someone in the top 25% of `played_hours`), age, gender, and experience can be used to predict a binary label response variable. In this case, the explanatory variables serve in our prediction model but also allow the stakeholders to better formulate targeted recruiting.

**Wrangling Plan:**

1. Will be creating a label `heavy_contributor` and will assign it a binary value of 1 given that the player is in the top quratile of contributors.
2. Will handle any missing cells for all explanatory variables and the response variable.
3. Will not be using the following variables `name` and `hashedEmail` and so will be dropping them to tidy the dataset further.
4. Will regroup non-Male/non-Female gender levels into `Other` to stabilize the variable.
5. Will split the data into training and testing data sets, with the split going 80:20 as this gives the model ~10 true positive cases of `heavy_contributor`.
6. Will stratify the training data using 5-fold cross-validation.

## Exploratory Data Analysis and Visualization

### Mean Value Calculations

In [ ]:
quantitative_variables <- players_data |>
    select(where(is.numeric))

mean_value_table <- quantitative_variables |>
    summarize(
        across(.cols = everything(),
               ~ round(mean(.x, na.rm = TRUE), 2))
    )

# quantitative_variables
mean_value_table

### Visualizations and Further Analysis

#### Heavy Contributors Distribution

In [ ]:
#the 75th percentile is about 0.6 hours = 26 minutes. (Q3)
quartile_3rd_cutoff <- players_data |>
    summarize(quartile_3rd = quantile(played_hours, 0.75, na.rm = TRUE)) |>
    pull(quartile_3rd)

# quartile_3rd_cutoff 

players_data_heavy <- players_data |>
    mutate(heavy_contributor = factor(played_hours >= quartile_3rd_cutoff,
                                      levels = c(FALSE, TRUE),
                                      labels = c("No", "Yes")))

# players_data_heavy

## Before Removing Players With Zero (0) Hours PLayed.
# First Histogram - Figure 1
heavy_histogram <- players_data_heavy |>
    ggplot(aes(x = played_hours)) +
    geom_histogram(
        binwidth = 0.25, 
        boundary = quartile_3rd_cutoff, 
        closed = "left") +
    geom_vline(
        xintercept = quartile_3rd_cutoff, 
        linetype = "dashed", 
        color = "red") +
    scale_x_continuous(
        # used a log scale to manage the tail.
        trans = "log1p",
        breaks = c(0, 1, 5, 10, 50, 100, 200))+
    annotate(
        "text", x = 1.05, y = 50, 
        label = "Heavy Contributor Cutoff (Q3)",
        hjust = 0, vjust = 0, colour = "red") +
    labs(
        x = "Total Play Hours (hrs)",
        y = "Number of Players",
        title = "Figure 1: Distribution of Total Play Hours With 3rd Quartile Cutoff") +
    theme_minimal(base_size = 12)

# Second Histogram - Figure 2
heavy_density_plot <- players_data_heavy |>
    ggplot(aes(x = played_hours)) +
    geom_density(fill = "lightblue", alpha = 0.8) + 
    geom_vline(
        xintercept = quartile_3rd_cutoff,  
        linetype = "dashed",  
        color = "red") +
    scale_x_continuous(
        trans = "log1p",
        breaks = c(0, 1, 5, 10, 50, 100, 200)) +
    annotate(
        "text", x = 1.05, y = 0.5, 
        label = "Heavy Contributor Cutoff (Q3)",
        hjust = 0, vjust = 0, colour = "red") +
    labs(
        x = "Total Play Hours (hrs)",
        y = "Density",  
        title = "Figure 2: Distribution of Total Play Hours With 3rd Quartile Cutoff") +
    theme_minimal(base_size = 12)

heavy_histogram
heavy_density_plot


- These distribution graphs (histograph and density plot) demonstrate the current recruitment problem that is faced by the stakehodlers, the vast majorty of players contribute nothing or close to nothing in play time and only few players contribute heavly towards the experiment. 

- The graphs illustrate a need for solving this issue and bridging the gap by aiming to fill the breaks between the local contribution areas (bars) and flatten the huge 0-play time spike skewing the graph towards one end over the other.

- The graphs also prompt me to experiment with removing the players with 0 played hours and recreate the graph to further examine the potential relationships present. 

In [ ]:
## After Removing Players With Zero (0) Hours PLayed.

# Players with 0 hours are removed:
players_data_0hrs_filtered <- players_data_heavy |>
    filter(played_hours > 0)

heavy_histogram_0hrs_removed <- players_data_0hrs_filtered |>
    ggplot(aes(x = played_hours)) +
    geom_histogram(
        aes(fill = after_stat(x >= quartile_3rd_cutoff)),
        binwidth = 1,
        boundary = quartile_3rd_cutoff,
        closed = "left",
        color = "grey"
        ) +
    scale_fill_manual(
        values = c("TRUE" = "#D81B60", "FALSE" = "#56B4E9"),
        labels = c("TRUE" = "Heavy Contributor", "FALSE" = "Other"),
        name = "Contributor Type"
        ) +
    scale_x_continuous(
        trans = "log1p",
        breaks = c(0, 1, 5, 10, 50, 100, 200)) +
    labs(
        x = "Total Play Hours (hrs)",
        y = "Number of Players", 
        title = "Figure 3: Play Time Distribution (Excluding Non-Participants)"
        ) +
    theme_minimal(base_size = 12)

heavy_histogram_0hrs_removed

#### Gender Category Regrouped

In [ ]:
players_data_heavy <- players_data_heavy |>
  mutate(
    gender_group = case_when(
        gender == "Male"   ~ "Male",
        gender == "Female" ~ "Female",
        # then group everything else, basically!
        TRUE               ~ "Other"),
    gender_group = factor(gender_group,
                          levels = c("Male", "Female", "Other")))

# players_data_heavy

#### Heavy Contributors Share By Group

##### By Gender:-

In [ ]:


# Histogram:
# To Show The Distribution of Heavy Contributors in Each Gender Grouping
heavy_contributor_by_gender_histo <- players_data_heavy |>
  ggplot(aes(x = gender_group, fill = heavy_contributor)) +
  geom_bar(color = "grey") +
  scale_fill_manual(
    values = c("Yes" = "#D81B60", "No" = "#56B4E9"),
    labels = c("Yes" = "Heavy Contributor", "No" = "Other"),
    name   = "Contributor Type"
  ) +
  labs(
    x     = "Gender Group",
    y     = "Number of Players",
    title = "Figure 4: Counts of Heavy and Other Contributors by Gender Group"
  ) +
  theme_minimal(base_size = 12)

heavy_contributor_by_gender_histo


**Evlauting Hypothesis After Reading Figure 4:**

While the **male group has by far the largest number of players overall and the lasrgest number of heavy contribuors in raw counts**, both the female and 'other' gender groups seem to have fewer total players but still include a substaintial heavy contributors player base. While Figure 4 helps me **understand which gender group is the most represented amongst heavy contriubtors (male)** it prompts me to **invistage the share/percentage of heavy contributors within each gender group to obtain a normalized view and find potential conversion problems** in certain groups (to agjust for group size difference.) - This question is further investigated through the next figure (Figure 5) below.

In [ ]:
heavy_contributor_by_gender <- players_data_heavy |>
    group_by(gender_group) |>
    summarize(contributor_total = n(),
            heavy_total = sum(heavy_contributor == "Yes"),
            heavy_per = round(100 * heavy_total / contributor_total, 2),
            .groups = "drop")

# heavy_contributor_by_gender

# Lollipop Plot:
# To show each gender's heavy contributors' share.
heavy_contributor_by_gender_plot <- heavy_contributor_by_gender |>
    ggplot(aes(x = gender_group, y = heavy_per)) +
    geom_segment(aes(xend = gender_group, y = 0, yend = heavy_per),
                 color = "#004D40", linewidth = 2) +
    geom_point(alpha = 0.8, size = 4, color = "#D81B60") +
    labs(
        title = "Figure 5: Proportion of Heavy Contributors Within Each Gender Group",
        x = "Gender Group", 
        y = "Heavy contributors Share (%)"
        ) +
    theme_minimal(base_size = 12)

heavy_contributor_by_gender_plot

**Quality of Contribution per Demographic:**

Despite the fact that the female group has a smaller playerbase, **the female gender group has the hgihest percentage of heavy contributors**, while the 'other' gender group shows a lower percentage compared to both male and female groupings. This plot illustrates that **it is far more likely that a female is a heavy contributor**. This gives my investigation some hints around some *problematic or less than optimal recruitement, targetting, and sampling strategies*. 

##### By Experience:-

In [ ]:
# Histogram
# Showing Experience Level vs Contribution 
heavy_contributor_by_experience_histo <- players_data_heavy |>
  ggplot(aes(x = experience, fill = heavy_contributor)) +
  geom_bar(color = "grey") +
  scale_fill_manual(
    values = c("Yes" = "#CC6677", "No" = "#DDCC77"),
    labels = c("Yes" = "Heavy Contributor", "No" = "Other"),
    name   = "Contributor Type"
      ) +
  labs(
    x     = "Experience Level",
    y     = "Number of Players",
    title = "Figure 6: Experience Distribution of Heavy and Other Contributors"
      ) +
  theme_minimal(base_size = 12)

heavy_contributor_by_experience_histo

**Composition of Contributors by Experience Level:**

While both amateur and vetern players dominate in terms of total numbers of players, there is a cleaer distinction when it comes to the number of heavy contributors. The plot shows us that **the largest heavy contributor player base is in the amateur subgroup** while the pro level contain the fewest number of heavy contributors. This suggests that the **largest pool of potential heavy contributors is within the amateur and vetern player bases**. Due to the fact that **recruitement resources are finite**, it might be better to focus on **precision rather than on recruiting volume**.

In [ ]:
heavy_contributor_by_experience <- players_data_heavy |>
    group_by(experience) |>
    summarize(contributor_total = n(),
            heavy_total = sum(heavy_contributor == "Yes"),
            heavy_per = round(100 * heavy_total / contributor_total, 2),
            .groups = "drop")

# heavy_contributor_by_experience

heavy_contributor_by_experience_plot <- heavy_contributor_by_experience |>
    ggplot(aes(x = experience, y = heavy_per)) +
    geom_segment(aes(xend = experience, y = 0, yend = heavy_per),
                 color = "#D35FB7", linewidth = 1.5) +
    geom_point(alpha = 0.8, size = 4, color = "#FFC107") +
    labs(
        title = "Figure 7: Heavy Contributors by Experience ",
        x = "Experience Level", 
        y = "Heavy contributors Share (%)"
        ) +
    theme_minimal(base_size = 12)

heavy_contributor_by_experience_plot

**Share of Heavy Contributors Within Each Experience Level:**

Despite having the second largest player base, the vetern subgroup shows a relatively lower heavy contribtor share (the lowest amongst the 5 subgroups). **Pros have the hgihest conversion rate to heavy contribtion** despire being the smallest group in aboslute numbers. This makes total sense as one would hypothesize that pros tend to spend more time gaming and therefore, if they are interested enough to help with the experiment, they would contribute more to the project. As a pro's **behavioral profile** might suggest an innate motivation or goal-driven approach to playing the game and therefore aligning with **consistent and heavy participation**. This suggests that **experience as a proxy for a behavioral profile makes it helpful in predicting heavy contribution status**.

Furthermore, taking another look at the plot shows that suprisingly and maybe countering the logic of the analysis of Figure 7 in the previous paragraph, **the beginner subgroup shows a high heavy contribution share**. One potential idea is that this could be due to the **novelty factor or high spare time availability**. This suggests that the researchers might want to take a look at **cofounding variables**. 

##### By Age:-

In [ ]:
heavy_contributor_by_age <- players_data_heavy |>
    group_by(Age) |>
    filter(!is.na(Age)) |>
    summarize(contributor_total = n(),
            heavy_total = sum(heavy_contributor == "Yes"),
            heavy_per = round(100 * heavy_total / contributor_total, 2),
            .groups = "drop")

# heavy_contributor_by_age
# By Age Plot:
heavy_contributor_by_age_plot <- heavy_contributor_by_age |>
    ggplot(aes(x = Age, y = heavy_per, color = heavy_per)) +
    geom_point(alpha = 0.8) +
    labs(
        x = "Age (years)",
        y = "Heavy Contributors Share (%)",
        title = "Figure 8: Share of Heavy Contributors by Age (Among Active Players)"
        ) +
    scale_color_gradientn(
        colors = c("#004D40", "#1E88E5", "#D81B60"),  # dark blue → blue → orange
        limits = c(0, 100),
        name = "Heavy share (%)"
        ) +
    theme_minimal(base_size = 12)

heavy_contributor_by_age_plot

**Analyzing The Probability of Heavy Contribution by Age:**

This graph captures the **intensity of heavy contribution within-age** for each age on the x-axis. There seems to be a discrepency for **much younger and much older** participants as they are **either highly engaged or absent** with almost no middle ground. This suggests a **binary behaviour for extreme ages**. It also suggests that these points **might act as outliers when training** the model and contribute to **biasing the model** one way or another so that **it cannot fit to generalize proprly**.

In [ ]:
age_hours_played_plot <- ggplot(players_data_heavy, aes(x = Age, y = played_hours, color = heavy_contributor)) +
    geom_point(alpha = 0.4) +
    geom_hline(
        yintercept = quartile_3rd_cutoff,
        linetype = "dashed",
        color = "black") +
    scale_y_continuous(
        trans = "log1p",
        breaks = c(0,1,5,10,50,100,200)) +
    labs(
        title = "Age vs Total Play Hours",
        x = "Age (years)", 
        y = "Total Play Hours (hours)",
        color = "Heavy Contributor?") +
    annotate(
        "text", x = 32, y = 0.8, 
        label = "Heavy Contributor Cutoff (Q3)",
        hjust = 0, vjust = 0, colour = "black") +
    theme_minimal(base_size = 12)

age_hours_played_plot

**Interperting The Relationship Between Age and Total Play Hours:**

Most heavy contributors (above the Q3 cutoff) cluster in the younger age bands (teens to mid-20s). These players dominate high play hours, reinforcing that younger participants are more likely to engage intensively with the experiment. **A clear demarcation betwen the behaviour of high contributors vs other type of participants**, this demarcation is highly visible (contrasted) in the same age group of younger participants, **suggesting that a deeper reason for a polarized engagement style** amongst younger players exists. The **plot strongly suggests a correlation between age and contribution and that contritbution is not uniformly distrubted accross ages**. 

## Methods and Plan

I plan to use linear regression to predict whether a player is likely to be a heavy contributor (`heavy_contributor` = response variable) to the project or not, heavy contributor defined as a binary result with yes meaning that the player is in the top-quartile of played_hours, from the following explanatory variables:
1. `Age`
2. `experience`
3. `gender_group`

Linear regression outputs a continuous score for each player, which we can interpret as their "likelihood" of being a heavy contributor. Linear regression is efficient even when we only have a few hundred players. Unlike k-NN, it doesn’t require a large amount of data to form accurate "neighborhoods." It’s also less prone to overfitting, which can be a risk when k-NN is used on small or sparse datasets. Linear regression has no major hyperparameters to tune, unlike k-NN which requires careful selection of the number of neighbors and scaling of all numeric variables. The model is quick to train, easy to validate, and fits well into a cross-validation framework where we can also pick the best classification threshold. We also reduce risk of information leakage by computing the heavy contributor threshold (the 75th percentile) using only the training set.

### Assumptions:

1. Each player is a separate unit (independent and tidy rows and variables = cell/observational unit). - we can check from the data descriptions section whether this is true or not
2. Proper linearity/scale for numeric predictors (we only have Age). - we can check by plotting/visually whether this numeric subset is skewed or not. 

### Limitations and Weaknesses:

1. Wide intervals and wrong estimates with very small groups, such as the gender category. This is the reason I proposed we merge less common gender levels to have "Male", "Female", and "Other" which will reduce uncertainty.
2. The model moght be more sensitive to outliers, in this data set, hypothetically if there is a player who is much older and is more experienced while at the same time putting more time in (heavy contributor) then the model might be influenced by this outlier and assume this on the path to a prediction.
3. Bias and confounding risks. If one subgroup is both underrepresented and systematically different, the model may implicitly learn biased associations.
4. Using the 75th percentile of played hours to define “heavy contributor” simplifies the problem to binary classification, but may obscure important information in the underlying play hour distribution.

### Compared to k-NN Classification:

| Criterion                       | Linear regression                                                        | k-NN classification                                                                                   |
|---------------------------------|--------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------|
| Prediction                      | Continuous score; classifies via a threshold.                            | Class label directly by majority vote amongst k neighbors.                                            |
| Fit to Question                 | Direct and simple baseline for a binary label                            | Native classifier for binary label.                                                                   |
| Interpretability                | Coefficients map to clear effects (Female vs Male, Pro vs Amateur, etc). | Local based and global behavior can be hard to summarize sometimes.                                   |
| Small-n performance (our n~196) | Data-efficient and stable                                                | Can have high variance and is general data hungry relative to linear regression, even with k tuning.  |


### Porposed Data Processing and Model Application

To ensure robust model training and fair performance evaluation, I will follow a structured workflow combining preprocessing, data splitting, and validation.

**Preprocssing and Data Cleaning:**

First, I will wrangle and clean my data by filtering out extreme and or irrelevant observations (e.g. from my experience with Figure 1 and Figure 2 and noticing the effects of having players with 0 `played_hours`, I will remove players with `played_hours = 0`). Furthermore, I will merge sparse categorical subgroups/levels (e.g. less represented and more sparse gender groups into "Other") to reduce noise and instability when training the model.

While workign on my EDA, Figure 7 was a perfect example of how to handle missing values. I will drop/remove missing values from my calculations. Furthermore, I will encovde factors such as gender_group and experience appropriately for modelling (for regression). 

Finally, I will define `heavy_contributor` as a binary outcome value based on the 75th percentile (Q3) of `played_hours`.

**Data Splitting:**

To reduce bias, I would used the locked model (i.e. the split will be done once at hte beginning and fixed throughout) for splitting the players dataset into training and testing (80:20, giving the training set about 10 true positive cases for `heavy_contributor`) to maintain class balance, this will insurance that there would be no information leakage. 

**Validation Strategy:**

Cross-validation is a resampling method used to assess how a predictive model will generalize to an independent dataset. It is especially important in this case since the dataset is limited to ~196 players, where holding out a large test set would reduce the data available for training. I will use 5-fold cross-validation. While in k-NN classifiers, cross-validation will primarly be used to tune the hyperparamtere k, in linear regression, cross-validation helps to verify model stability across different subsets of the data, quantify any generalization error and reduce the risk of overfitting. 

**Training and Evaluation:**
1. Fitting: I will fit the lienar regression model using the training data to predict the probability of being a `heavy_contributor` as a function of `Age`, `experience` and `gender_group`.
2. Assesment: Evaluation will be done using metrics such as accuracy, precision, recall, and RSME.